In [ ]:
""" Psychometric stuff.

# See notes here:
# - Evernote: https://www.evernote.com/shard/s101/nl/11215139/42869e39-d0af-4935-a285-46236ef79adc?title=Psychometric
(hav elooked thru, nothing missing that should be here).

"""

In [ ]:
from pythonlib.tools.plottools import savefig
from pythonlib.dataset.dataset_analy.primitives import *
from pythonlib.dataset.dataset_preprocess.primitives import *
from pythonlib.dataset.dataset import Dataset, load_dataset, load_dataset_daily_helper
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

%load_ext autoreload
%autoreload 2


In [ ]:

# Chars
# animal = "Pancho"
# date = "230127"

animal = "Diego"
date = "231211"

D = load_dataset_daily_helper(animal, date)


### How to represent task so that can easily cluster into psychometric groups

In [ ]:
T.PlanDat.keys()

In [ ]:
# set of base-level prims and their locations
T.PlanDat["Plan"]

In [ ]:
T = D.Dat.iloc[ind]["Task"]
T.get_shapes_hash()

In [ ]:
# Distance metric for comparing characters
# - first, between prims (score for locaiton and shape distnace

# Or simply use pixel distance, after transforming one to maximally algin with the other. 




In [ ]:
D.strokes2image("strokes_task")

### old code From analy_psychometric notebook

In [ ]:
sdir = f"/data2/analyses/main/psychometric/monkey/{animal}-{expt}/plots"
os.makedirs(sdir, exist_ok=True)
from pythonlib.dataset.psychometric import plotMultTaskcategories
for max_n_per_grid in [None] + list(range(5)):
    sdirthis = f"{sdir}/npergrid_{max_n_per_grid}"
    os.makedirs(sdirthis, exist_ok=True)
    plotMultTaskcategories(D, sdir=sdirthis, max_n_per_grid=max_n_per_grid)

In [ ]:
# each task extract its number


# update grid plot code to work with strokes list

#### 2) compute some scalar score

### NEW METHOD - using shapes and their locations

In [ ]:
# 1) Get graph represnetation of each char

# Extract base prims for each char
ind = 403
ind2 = 404
# D.Dat[""]
list_shloc1 = D.taskclass_shapes_loc_configuration_extract(ind, loc_version="pixel")["shape_loc"]
list_shloc2 = D.taskclass_shapes_loc_configuration_extract(ind2, loc_version="pixel")["shape_loc"]

D.plotMultTrials2([ind, ind2])
D.plotMultTrials2([ind, ind2], "strokes_task")


print(list_shloc1)
print(list_shloc2)
# 2) Find related chars

In [ ]:


def _dist_sh(sh1, sh2):
    """
    Dist (from 0 to 1) between these shape strings. counts up how many
    mathces out fo the 4 substrings, and then norms to 0 and 1.
    sh = 'line-11-1-0'
    """
    from pythonlib.tools.stringtools import decompose_string
    
    NTOT = 4
    
    sh1_list = decompose_string(sh1, "-")
    sh2_list = decompose_string(sh2, "-")
    # print(sh1)
    # print(sh1_list)
    # assert False
    assert len(sh1_list)==NTOT
    assert len(sh2_list)==NTOT
    
    nmatch = sum([x==y for x,y in zip(sh1_list, sh2_list)])
    
    # normalize 
    dist = 1 - nmatch/NTOT
    
    return dist
    
# _dist_sh(shloc1[0][0], shloc1[6][0])


DIAG = D.sketchpad_compute_diagonal_using_all_strokes()

def _dist_loc(loc1, loc2):
    """ Eucl dist between two locations, normalized to 0 and 1
    - loc, (2,) array.
    RETURNS:
    - dist, scalar, normalized between 0 (same) and 1 (sketchpad diag).
    """
    
    if isinstance(loc1, list):
        loc1 = np.array(loc1)
        loc2 = np.array(loc2)
        
    d_eucl = np.linalg.norm(loc1 - loc2)
    
    # normalize to max.
    dist = d_eucl/DIAG
    
    return dist

def _dist_shloc(shloc1, shloc2):
    """
    - shloc, (sh string, loc xy)
    """
    d1 = _dist_sh(shloc1[0], shloc2[0])
    d2 = _dist_loc(shloc1[1], shloc2[1])
    return np.mean([d1, d2])
    

In [ ]:
# Given two shape_loc configs, compute their distance
# for each (shape,loc) object in one task, get its min distance to all in other task.

def _dist(list_shloc1, list_shloc2):
    """ Distance between two shape/loc objects
    - shloc = (shape string, loc num(2,1))
    """
            
    distmat = np.asarray([[_dist_shloc(shloc1, shloc2) for shloc2 in list_shloc2] for shloc1 in list_shloc1]) # (len1, len2)

    # get mean across both dimensions
    d1 = np.mean(np.min(distmat, axis=0))
    d2 = np.mean(np.min(distmat, axis=1))
    dall = np.mean([d1, d2])
    
    return dall


In [ ]:
SDIR = D.make_savedir_for_analysis_figures_BETTER("char_psycho")
sdir = f"{SDIR}/best_matches_for_each_char"
import os
os.makedirs(sdir, exist_ok=True)
print(sdir)

In [ ]:
# include only chars
Dc = D.copy()
Dc.Dat = Dc.Dat[Dc.Dat["task_kind"]=="character"].reset_index(drop=True)
Dc.preprocessGood(params=["no_supervision", "remove_online_abort"])

In [ ]:
list_char = Dc.Dat["character"].unique().tolist()
inds, list_char = Dc.taskcharacter_extract_examples(list_char)

In [ ]:
# first collect pairwise between all chars.
# Then make plots using that info.

# get map from ind to score
map_char_listshloc = {}

for ind, char in zip(inds, list_char):
    list_shloc = Dc.taskclass_shapes_loc_configuration_extract(ind, loc_version="pixel")["shape_loc"]
    map_char_listshloc[char] = list_shloc


In [ ]:

distances = {}
for i in range(len(inds)):
    print(i)
    for j in range(i+1, len(inds)):
        ind1 = inds[i]
        char1 = list_char[i]
        
        ind2 = inds[j]
        char2 = list_char[j]
        
        list_shloc1 = map_char_listshloc[char1]
        list_shloc2 = map_char_listshloc[char2]
        
        distances[(i, j)] = _dist(list_shloc1, list_shloc2)
        


In [ ]:
# ni = np.max(np.asarray(list(distances.keys())), axis=0)[0]
# nj = np.max(np.asarray(list(distances.keys())), axis=0)[1]

ni = len(list_char)
nj = len(list_char)

# plot in heatmap
distances_mat = np.zeros((ni, nj))
for i in range(ni):
    
    # Assymetric
    # for j in range(i+1, nj):
    #     distances_mat[i, j] = distances[(i, j)]
    
    # Symmetric
    for j in range(nj):
        if j==i:
            distances_mat[i, j] = 0.0001
        elif j>i:
            distances_mat[i, j] = distances[(i, j)]
        elif j<i:
            distances_mat[i, j] = distances[(j,i)]
        else:
            assert False
assert np.any(distances_mat==0.)==False, "sanity check that modified all slots"

fig, ax = plt.subplots()
im = ax.imshow(distances_mat)
plt.colorbar(im)


In [ ]:

# Given a single char, get its score against all other char
i=103
list_score = distances_mat[i,:]
charthis = list_char[i]
print(charthis)
Dc._taskcharacter_find_plot_sorted_by_score(list_char, list_score, True, "/tmp", 1, 20, "test")


In [ ]:
Dc.Dat["task_stagecategory"].unique()

In [ ]:


# list_task_stagecategory = ["charstrokeseq-ss-49"]
list_task_stagecategory = ["charstrokeseq-ss-65", "charstrokeseq-ss-64", "charstrokeseq-ss-63", "charstrokeseq-ss-66"]

In [ ]:
# Go thru all chars
nsubplots = 20

for i, charthis in enumerate(list_char):
    list_score = distances_mat[i,:]
    print(charthis)
    
    # Check if char is in desired list of cats
    catthis = Dc.Dat[Dc.Dat["character"]==charthis]["task_stagecategory"].tolist()[0]
    
    if catthis in list_task_stagecategory:
        
        Dc._taskcharacter_find_plot_sorted_by_score(list_char, list_score, True, sdir, 1, nsubplots, charthis,
                                                    plot_which="worst")
        plt.close("all")

##### Check that the prims extracted across the dataset are reasonable

In [ ]:
D.taskclass_shapes_extract_unique_alltrials()

In [ ]:
# TODO:
# Account for the centering, this leads to incorrect location scores.
# Should use relational coords?
# Or graph representation?
# ---> Best: do realignement of some sort... using image?

In [ ]:
D.plotSingleTrial(ind1)

In [ ]:
# Represent prim using the (shape, scale, location of onset relative to center).

Prim = tokens[0]["Prim"]
Prim.extract_as()

In [ ]:
Prim.label_classify_prim()

In [ ]:
Tt = T.extract_monkeylogic_ml2_task()
Tt.get_tasknew()["Plan"].keys()

In [ ]:
Tt.get_tasknew()["Plan"]["PrimsExtraParams"]


In [ ]:
# 1) check all tasks to see if abstract transforms are occuring.
# 2) in computing angle, also compute to offset. 

In [ ]:
Prim.ParamsAbstract

In [ ]:
Prim.ParamsConcrete

In [ ]:
# example cahracters to compare, same psycho
task = "charstrokeseq-62-28-948510"
task = "charstrokeseq-62-7-372727"
D.Dat[D.Dat["character"]==task].index

In [ ]:
665

In [ ]:
ind = 405
T = D.Dat.iloc[ind]["Task"]
T.PlanDat["RelsAttachIndicesAlongStrok_RelInd"]
T.PlanDat["PrimsAsStrings"]

In [ ]:
### snaity check, plot tasks and overlay sh_loc on image

import random
inds = random.sample(range(len(D.Dat)), 20)

D.plotMultTrials2(inds)
fig, axes, idxs = D.plotMultTrials2(inds, "strokes_task")

for ax, ind in zip(axes.flatten(), inds):
    print("---------", ind)
    # T = D.Dat.iloc[ind]["Task"]
    # T.PlanDat["RelsAttachIndicesAlongStrok_RelInd"]
    print(T.PlanDat["CentersActual"])
    list_shloc = D.taskclass_shapes_loc_configuration_extract(ind, loc_version="pixel")["shape_loc"]
    print(list_shloc)
    
    for i, shloc in enumerate(list_shloc):
        sh = shloc[0]
        loc = shloc[1]
        ax.plot(loc[0], loc[1], "or")
        t = f"{i}_{sh}"
        ax.text(loc[0], loc[1], t, fontsize=6, color="b")



In [ ]:
%matplotlib inline
inds = [665, 709]

D.plotMultTrials2(inds)
fig, axes, idxs = D.plotMultTrials2(inds, "strokes_task")

for ax, ind in zip(axes.flatten(), inds):
    print("---------", ind)
    # T = D.Dat.iloc[ind]["Task"]
    # T.PlanDat["RelsAttachIndicesAlongStrok_RelInd"]
    print(T.PlanDat["CentersActual"])
    list_shloc = D.taskclass_shapes_loc_configuration_extract(ind, loc_version="pixel")["shape_loc"]
    print(list_shloc)
    
    for i, shloc in enumerate(list_shloc):
        sh = shloc[0]
        loc = shloc[1]
        ax.plot(loc[0], loc[1], "or")
        t = f"{i}_{sh}"
        ax.text(loc[0], loc[1], t, fontsize=6, color="b")



In [ ]:
D.sequence_tokens_clear_behclass_and_taskclass()

In [ ]:
D.behclass_preprocess_wrapper(True, False)

In [ ]:
##### Get a trial's tokens and compare to plot

In [ ]:
# [GOOD] Delete all tokens for this trial
if False:
    T = D.Dat.iloc[ind]["Task"]
    T._tokens_delete()
    
    Beh = D.Dat.iloc[ind]["BehClass"]
    Beh.Alignsim_Datsegs = None
    Beh.Alignsim_Datsegs_BehLength = None

In [ ]:
ind = 403
T = D.Dat.iloc[ind]["Task"]

In [ ]:
tokens = D.taskclass_tokens_extract_wrapper(ind, "task")
D.plotSingleTrial(ind, task_add_num=True);


In [ ]:
T.plotStrokes(ordinal=True)
for i, t in enumerate(tokens):
    print(i, t["shape"], t["center"])

In [ ]:
D.behclass_preprocess_wrapper(reset_tokens=True, skip_if_exists=False)

In [ ]:
T.primitives_extract_final()

In [ ]:
D.taskclass_shapes_loc_configuration_extract(ind, loc_version="pixel")["shape_loc"]

In [ ]:
T = D.

In [ ]:
T.PlanDat.keys()

### [BELOW IS OLD] Psychometric, taken from analy_dataset_summarize

##### Rename tasks based on Plan. 

This allows matching tasks that have same abstract plan but different size, for eg

In [ ]:
# Generate all beh class
D.behclass_generate_alltrials()

In [ ]:
# Try a single trial
ind = 100
Beh = D.Dat.iloc[ind]["BehClass"]
Task = D.Dat.iloc[ind]["Task"]
Task.PlanDat["Plan"]
plan = Task.PlanDat["Plan"]


In [ ]:

# convert plan to a hashable

def _list_to_hashable(inlist):
    
    outlist = []
    for i, this in enumerate(inlist):
        if isinstance(this, list):
            outlist.append(_list_to_hashable(this))
#             inlist[i] = _list_to_hashable(this)
        else:
            outlist.append(str(this))
#             inlist[i] = str(this)
    return tuple(outlist)

_list_to_hashable(plan)


        

In [ ]:
# Change all character names to 
from pythonlib.tools.pandastools import applyFunctionToAllRows
def F(x):
    Task = x["Task"]
    plan = Task.PlanDat["Plan"]
    return hash(_list_to_hashable(plan))

D.Dat = applyFunctionToAllRows(D.Dat, F, "planhash")

In [ ]:
# Extract a single plan based on hash
print(D.Dat["planhash"].value_counts())

inds = D.Dat[D.Dat["planhash"]==4457446285971416165].index.tolist()
D.plotMultTrials2(inds)
D.plotMultTrials2(inds, "strokes_task")



In [ ]:
sdir = f"{SAVEDIR_FIGS}/using_planhash/quick"
os.makedirs(sdir)

In [ ]:
# sort trials by size of grid
def _get_grid_spacing(ind):
    """ Get scalar spacing for grids, for this trial of D.Dat
    NOTE: is quick and dirty, just gets the diff of centers,looking at x.
    doesnt work if tasks have diff size due to cell size (not due to centers).
    """
    Task = D.Dat.iloc[ind]["Task"]
    par = Task.get_grid_params()
    return par["grid_diff_x"]

_get_grid_spacing(100)


In [ ]:
# For each planhash with > N trials, plot all the trials, sorted by something

# 1. get all planhash with > N trials
N = 5
x = D.Dat["planhash"].value_counts()
list_planhash = [ph for ph, counts in zip(x.index, x.values) if counts > N]

print("This many tasks :", len(list_planhash))

for planhash in list_planhash:
    # get all trials for this ph
    inds = D.Dat[D.Dat["planhash"]==planhash].index.tolist()
    
    # sort by grid spacing size
    x = [(i, _get_grid_spacing(i)) for i in inds]
    x = sorted(x, key=lambda x: x[1])
    inds = [xx[0] for xx in x]
    sizes = [xx[1] for xx in x]
    
    # Plot
    fig1, _, _ = D.plotMultTrials(inds, "strokes_beh", color_by="order")
    fig2 = D.plotMultTrials(inds, "strokes_task", titles=sizes)
    
    # save
    fig1.savefig(f"{sdir}/alltrials-chron-{planhash}-beh.pdf")
    fig2.savefig(f"{sdir}/alltrials-chron-{planhash}-task.pdf")
    

##### [Psychometric] Categorize tasks based on their psychonmetric groupings

In [ ]:
# get a task for testing
Task = D.Dat.iloc[0]["Task"]
taskthis = Task.PlanDat["Plan"] # look for tasks similar to this one

In [ ]:
# HACK - reassing train test for motifchar3 based on fixed tasks

def F(x):
    if x["random_task"]:
        return "train"
    else:
        return "test"
    
D.Dat = applyFunctionToAllRows(D.Dat, F, "monkey_train_or_test")

In [ ]:
# Find task groupgings


def check_tasks_same_psychometric_group(task, taskother, row_diff=3):
    """ if taskother is in same psychometric group as task, then
    returns True.
    Detects bsaed on all plan being identical except a speciifc relatin
    (curerntl yhard coded for motifchar3
    PARAMS:
    - task, taskother, TaskGeneral instances
    """
    
    def _check_row_same(row1, row2):
        """ recusrsive, check tuples are same
        """
        return row1==row2
#         if isinstance(row1, tuple):
#             if not isinstance(row2, tuple):
#                 return False
#             for r1, r2 in zip(row1, row2):
#                 return _check_row_same(r1, r2)
#         else:
#             return row1==row2
            
    
    # 1) convert tasks to hashable plans
    plan1 = _list_to_hashable(task.PlanDat["Plan"])
    plan2 = _list_to_hashable(taskother.PlanDat["Plan"])

    # 2) check item by item
    list_plan_common_rows = []
    for i, (row1, row2) in enumerate(zip(plan1, plan2)):
        if i==row_diff:
            # then allow to be diff
            # row1 =  ['translate_xy',
#                   array(-1.),
#                   ['interp_end1_end2_99', 'end2', array([0., 0.])]],
            tmp = [] # to collect this row elements ignoring the psycho thing
            for ii, (rowrow1, rowrow2) in enumerate(zip(row1, row2)):
                if ii==2:
                    # rowrow1 = ['interp_end1_end2_99', 'end2', array([0., 0.])]
                    # ignore the first item, interp_end1_end2_99
                    x1 = rowrow1[1:]
                    x2 = rowrow2[1:]
                else:
                    x1 = rowrow1
                    x2 = rowrow2
                same = _check_row_same(x1, x2)
                if not same:
                    return False, None
                tmp.append(x1)
            list_plan_common_rows.append(tuple(tmp))
        else:
            # check if same
            same = _check_row_same(row1, row2)
            if not same:
                return False, None
            list_plan_common_rows.append(row1)
    return True, tuple(list_plan_common_rows)

def get_tasks_psycho_group(Task):
    """ Get this tasks group. 
    Assumes a specific kind of manipualtion of plan..
    PARAMS;
    - Task, TaskClassGeneral
    RETURNS:
    - list_tasks, list_inds, list_interps, list_plancommon, all same length.
    """
    row_diff = 3
    list_tasks = []
    list_inds = []
    list_interpstrings = []
    list_plancommon = []
    for i in range(len(D.Dat)):
        if D.Dat.iloc[i]["monkey_train_or_test"]=="train":
            continue
        TaskOther = D.Dat.iloc[i]["Task"]    
        same, list_plan_common_rows = check_tasks_same_psychometric_group(Task, TaskOther, row_diff)
        if same:
#             print("Found same task, row: ", i)
            list_tasks.append(TaskOther)
            list_inds.append(i)
            list_plancommon.append(list_plan_common_rows)

            # Pull out its relation value
            interp_string = TaskOther.PlanDat["Plan"][row_diff][2][0] # interp_end1_end2_99
            list_interpstrings.append(interp_string)

    print(list_interpstrings)
    list_interps = [int(x[-2:]) for x in list_interpstrings]
    list_plancommon_hash = [hash(x) for x in list_plancommon]
    assert len(set(list_plancommon_hash))==1, "shuld all be same psycho group..."
    
    return list_tasks, list_inds, list_interps, list_plancommon

In [ ]:
# Get psychometric groyup for all tasks

out = [] # list of dict, index: group
dftest = D.Dat[D.Dat["monkey_train_or_test"]=="test"]

def _exists(ind, out):
    """ Check if this trial ind is present alreadry in out"""
    for x in out:
        if x["ind"]==ind:
            return x, True
    return None, False

for i in dftest.index.tolist():
    print(i)
    Task = D.Dat.iloc[i]["Task"]
    list_tasks, list_inds, list_interps, list_plancommon = get_tasks_psycho_group(Task)
    
    # assign groups to tasks. if already assigned, confirm that is same as before
    for ind, interp, plan in zip(list_inds, list_interps, list_plancommon):
        # Check if exists
        x, exists = _exists(ind, out)
        if exists:
#             print("Exists...")
            assert x["interp"]==interp
            assert x["plan_hash"]==hash(plan)
        else:
#             print("Doesnt exist, adding this task")
            # append
            out.append({
                "ind":ind,
                "interp":interp,
                "plan_hash":hash(plan),
                "plan_common":plan
            })
            
# TODO: to speed up, dont only look for a tasks group if it is not already in out. Only
# do this if sure that this code doesnt mistakenly assign assign a task to multiple groups.

In [ ]:
# Check that all test tasks have been assigned. If so, then confident that each task is assigned to one and only one.
for i in dftest.index.tolist():
    assert _exists(i, out)[1]

In [ ]:
# sort out
out = sorted(out, key=lambda x: x["ind"])


In [ ]:
dfpsycho = pd.DataFrame(out)

In [ ]:
# Print some stats about psycho groups
list_groups = dfpsycho["plan_hash"].unique().tolist()
print("-- found this many psycho groups: ", len(list_groups))

for grp in list_groups:
    dfthis = dfpsycho[dfpsycho["plan_hash"]==grp]
    print(grp, " : ", len(dfthis))
    
print("-- With these num samples per interp")
for grp in list_groups:
    dfthis = dfpsycho[dfpsycho["plan_hash"]==grp]
    print(grp)
    print(dfthis["interp"].value_counts())
    


In [ ]:
dfthis.sort_values("interp")

In [ ]:

sdir = f"{SAVEDIR_FIGS}/using_planhash/psycho_quick"
os.makedirs(sdir)


In [ ]:
# Plot each group
for grp in list_groups:
    dfthis = dfpsycho[dfpsycho["plan_hash"]==grp]
    # sort by psychometric
    dfthis = dfthis.sort_values("interp")
#     print(dfthis)
    inds = dfthis["ind"].tolist()
    interps = dfthis["interp"].tolist()
#     print(inds)
    fig1 = D.plotMultTrials(inds, "strokes_task", titles=interps);
    fig2, _, _ = D.plotMultTrials(inds, color_by="order");
    
    fig1.savefig(f"{sdir}/alltrials-group_{grp}-task.pdf")
    try:
        fig2.savefig(f"{sdir}/alltrials-group_{grp}-beh.pdf")
    except:
        fig2.savefig(f"{sdir}/alltrials-group_{grp}-beh.png")
    
    plt.close("all")


In [ ]:
fig2, _, _ = D.plotMultTrials(inds, color_by="order");
fig2.savefig(f"{sdir}/alltrials-group_{grp}-beh.jpg")


In [ ]:
# Give this group a name, based on the common plan elements


In [ ]:
D.plotMultTrials(list_inds, "strokes_task", titles=list_interps);
D.plotMultTrials(list_inds, color_by="order");
                                  